<table>
    <tr>
        <td>
            <img src="extra/logo_scgis.png" width="100" height="100" />
        </td>
        <td>
            <center>
                <font size = 1>
                8 НОЯБРЯ 2020<br>
                АНТИКОНФЕРЕНЦИЯ СООБЩЕСТВА ПРИРОДООХРАННЫХ ГИС В РОССИИ<br><br>
                </font>
                <font size = 3>
                Материалы к вебинару<br> <b>Получение и обработка данных дистанционного зондирования с помощью Python:<br>строим простую систему мониторинга окружающей среды в Jupyter Notebook.</b><br><br>
                </font>
                <font size = 2>
                <b>Эдуард Казаков</b><br>
                NextGIS, Государственный гидрологический институт
                </font>
            </center>
        </td>
    </tr>
</table>

# Преамбула

Эти материалы призваны познакомить интересующихся с некоторыми аспектами использования языка Python для решения задач, связанных с организацией мониторинга окружающей среды на базе открытых данных дистанционнго зондирования Земли из космоса. Jupyter - удобная среда для демонстрации основных идей, но есть множество других способов использовать приведенный код.

Несмотря на появление множества потрясающих инструментов, таких как Google Earth Engine, умение запрограммировать собственную цепочку обработки остаётся очень ценным навыком. У внешних платформ всегда будут ограничения по доступным данным и набору инструментов обработки, а в своём собственном коде вы ограничены только временем своей жизни на разработку :)

Рассматриваемая задача достаточно условна, но включает работу с распространенными данными и способами взаимодействия с ними.

&#128151; Подготовлено специально для прекрасных людей, связанных с сообществом природоохранных ГИС или сочувствующих ему, в 2020 году. 

# Постановка задачи
Мониторинг - фундаментальный и понятийно очень простой процесс, лежащий в основе взаимодействия человека и окружающего мира. Говоря простым языком, под мониторингом можно понимать любое периодическое наблюдение за состоянием отдельного природного объекта или природного комплекса. Например, если вы каждое утро спускаетесь к реке и по некоторой шкале оцениваете её мутность, ведя историю таких замеров, с чистой совестью можно всем вокруг рассказывать, что вы осуществляете мониторинг окружающей среды - и это будет правдой.

Наступление эпохи дистанционного зондирования открыло огромные мониторинговые возможности, ведь теперь за планетой непрерывно наблюдают сотни космических аппаратов. В зависимости от используемых источников, каждый месяц, неделю, день, а иногда и чаще, мы получаем цифровой отпечаток состояния любого уголка Земли. Накопленные за вот уже полувековую историю космических наблюдений данные перевернули наше представления о родной планете, а главное - каждую секунду эти знания продолжают пополняться.

Благодаря открытым программам ДЗЗ (<a href="https://www.copernicus.eu/en">ESA Copernicus</a>, <a href="https://eospso.nasa.gov/content/nasas-earth-observing-system-project-science-office">NASA EOS</a> и многие другие) бесплатный доступ к постоянно обновляемым архивам космической съемки имеет всё человечество (подключенное к сети Интернет). А это значит, что даже самые частные задачки, связанные с периодическим наблюдением за состоянием поверхности, могут быть хотя бы в первом приближении решаемы любым человеком в любой части мира, будь то исполненный надежд школьник или суровый лесохозяйственник.

Мы попробуем сделать простую и минималистичную "систему мониторинга", которая при каждом запуске будет получать свежие данные ДЗЗ на выбранную территорию, немного их обрабатывать и представлять результаты в виде человекочитаемых сводок: статистических, графических и картографических. 

## Данные

В качестве источников данных выберем два:
* <a href="https://sentinel.esa.int/web/sentinel/missions/sentinel-2">Sentinel 2</a> - наиболее популярные сегодня данные с относительно высоким пространственным разрешением. Приблизительно каждые 5 суток на любую часть планеты мы получаем такие данные с детальностью около 10 метров на пиксель. Это очень хорошее качество. 
* <a href="https://modis.gsfc.nasa.gov/about/">MODIS</a> - Данные MODIS позволяют получать ежедневные данные с разрешением от 250 метров на пиксель. Несмотря на вроде бы низкое разрешение (на самом деле нет!), эти данные незаменимы для мониторинга динамических масштабных процессов, таких как лесные пожары, крупные наводнения, дрейф морских льдов и так далее.

Выбор также обусловлен методическими целями. Дело в том, что способ получения у этих двух источников данных весьма различен. Для получения данных Sentinel мы воспользуемся программным интерфейсом (API), то есть будем формировать специальные запросы к официальному шлюзу, который в ответ будет предлагать нам загрузить подходящие данные. А для получения данных MODIS мы будем идти в специальное файловое хранилище, где по имени файла сами будем разбираться, что нужно загрузить.

## Задача

Мы хотим следить за динамикой значений вегетационного индекса <a href="https://gis-lab.info/qa/ndvi.html">NDVI</a> в пределах некоторого участка территории. Участок территории определяется содержимым векторного слоя.

При каждом запуске должны догружаться новые данные, маскироваться облачность, рассчитываться NDVI, извлекаться данные в пределах участка.

Для каждого участка на даты наличия космических снимков мы хотим уметь увидеть отчёт:
* Среднее, минимальное и максимальное значения NDVI
* График динамики NDVI за всю накопленную историю наблюдений
* Карту распределения для последних данных поверх картографической подложки

### Импортируем библиотеки, возможности которых нам будут необходимы

In [40]:
# Инструменты для работы с файловой системой
import os

# Библиотеки для работы с векторными и растровыми данными
import gdal, ogr

# Библиотека для взаимодействия с интернет-ресурсами
import requests

# Библиотека для разбора XML документов
import xml.etree.ElementTree as etree

# Библиотека для работы с датами и временем
from datetime import datetime

### Для начала подготовим функции, которые позволят как-нибудь легко и просто загружать данные.

Начнём с данных Sentinel-2. Официально они распространяются с помощью портала <a href="https://scihub.copernicus.eu/">The Copernicus Open Access Hub</a>, который предоставляет различные способы получения данных, в том числе - через графический веб-интерфейс. Но это нам не подходит - нам нужна автоматизация. По этому случаю есть официальное <a href="scihub.copernicus.eu/userguide/8BatchScripting">API</a>, то есть способ получать данные программно. Это HTTP API, то есть оно принимает запросы по протоколу HTTP, а слать такие запросы может и обычный веб-браузер. Попробуйте, к примеру, обратиться по адресу <a href="https://scihub.copernicus.eu/dhus/search?q=*&rows=25">https://scihub.copernicus.eu/dhus/search?q=*&rows=25</a> и ввести логин/пароль.

Вы получите ответ на запрос в машиночитаемом формате XML. А запрос подразумевал информацию о последних 25 сценах, добавленных в систему, без дополнительных указаний. Можно добавлять разные отдельные условия, сужающие запрос. Например, вот так <a href="https://scihub.copernicus.eu/dhus/search?q=*&rows=25&platformname=Sentinel-2">https://scihub.copernicus.eu/dhus/search?q=*&rows=25&platformname=Sentinel-2</a>, дополнительно указав параметр platformname, мы получим только описание сцен со спутников Sentinel-2

Значит, нам нужно написать код, который будет формировать такой запрос и разбирать полученные данные. Для начала - эксперименты!

In [78]:
# Пробуем просто выполнить такой запрос
r = requests.get('https://scihub.copernicus.eu/dhus/search?q=(platformname:Sentinel-2)&rows=25', auth=('spbgeotex','SpbGeoTex0'))

In [53]:
type(r)

requests.models.Response

In [54]:
# что нам вернулось? Код статуса
r.status_code

200

In [55]:
# содержимое, первая тысяча символов
r.content[0:1000]

b'<?xml version="1.0" encoding="utf-8"?><feed xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/" xmlns="http://www.w3.org/2005/Atom">\n<title>Sentinels Scientific Data Hub search results for: platformname:Sentinel-2</title>\n<subtitle>Displaying 0 to 24 of 22831334 total results. Request done in 0 seconds.</subtitle>\n<updated>2020-11-05T17:20:53.714Z</updated>\n<author>\n<name>Sentinels Scientific Data Hub</name>\n</author>\n<id>https://scihub.copernicus.eu/dhus/search?q=platformname:Sentinel-2</id>\n<opensearch:totalResults>22831334</opensearch:totalResults>\n<opensearch:startIndex>0</opensearch:startIndex>\n<opensearch:itemsPerPage>25</opensearch:itemsPerPage>\n<opensearch:Query role="request" searchTerms="platformname:Sentinel-2" startPage="1"/>\n<link rel="self" type="application/atom+xml" href="https://scihub.copernicus.eu/dhus/search?q=platformname:Sentinel-2&amp;start=0&amp;rows=25"/>\n<link rel="first" type="application/atom+xml" href="https://scihub.copernicus.eu/dhus/se

Это XML-код. Нам нужно его превратить во что-то вменяемое, с чем можно было бы работать средствами Python. Для этого хорошо подходит стандартная библиотека xml

In [13]:
# читаем весь пришедший текст как XML-дерево
answer_data = etree.ElementTree(etree.fromstring(r.content)).getroot()

# Ищем в нём все отдельные записи
entries = answer_data.findall('{http://www.w3.org/2005/Atom}entry')

In [14]:
# что за записи?
entries

[<Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5E4188>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5EFBD8>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5EE638>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A6EE958>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5DE818>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5A2048>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5B29F8>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5B6408>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5D0CC8>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A5D1CC8>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A6AD048>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A6C3F98>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A6CADB8>,
 <Element '{http://www.w3.org/2005/Atom}entry' at 0x000002D82A6BCC78>,
 <Elem

In [16]:
# возмём конкретную запись и посмотрим, какие есть поля
for child in entries[0]:
    print (child.get('name'))

None
None
None
None
None
None
beginposition
endposition
ingestiondate
orbitnumber
relativeorbitnumber
vegetationpercentage
notvegetatedpercentage
waterpercentage
unclassifiedpercentage
mediumprobacloudspercentage
highprobacloudspercentage
snowicepercentage
cloudcoverpercentage
level1cpdiidentifier
gmlfootprint
footprint
format
processingbaseline
platformname
filename
instrumentname
instrumentshortname
size
s2datatakeid
producttype
platformidentifier
orbitdirection
platformserialidentifier
processinglevel
identifier
uuid


In [17]:
# Если нас интересует конкретное поле, получаем его значение
for child in entries[0]:
    if (child.get('name') == 'uuid'):
        print ('UUID: %s' % child.text)
        
    if (child.get('name') == 'beginposition'):
        print ('Begin date: %s' % child.text)

Begin date: 2020-11-05T08:51:39.024Z
UUID: 9453d637-e0c8-4d12-accc-c20d7f59e567


In [20]:
# Получили список, а дальше?
# А дальше нужно скачать! Скачивание производится на базе UUID.
# Базовый адрес для скачивания у портала:
odata_base_url = 'https://scihub.copernicus.eu/dhus/odata/v1/'

# будем скачивать uuid 9453d637-e0c8-4d12-accc-c20d7f59e567
uuid = '9453d637-e0c8-4d12-accc-c20d7f59e567'

# формируем адрес для загрузки: прибавляем к базову адресу uuid в требуемой обёртке
downloader_string = odata_base_url + 'Products(\'%s\')/$value' % str(uuid)

# посмотрим на адрес получившийся
print(downloader_string)

https://scihub.copernicus.eu/dhus/odata/v1/Products('9453d637-e0c8-4d12-accc-c20d7f59e567')/$value


In [29]:
# Подключаемся к этому адресу как к потоку данных
r = requests.get(downloader_string, auth=('spbgeotex','SpbGeoTex0'), stream=True)

In [30]:
r.status_code

200

In [31]:
# Этой переменной будем отслеживать сколько уже скачалось
current_size = 0

# Будем качать кусками по 4096 бит
chunk_size = 4096

# Файл, куда качаем
test_file = 'data/test.zip'

with open(test_file, 'wb') as fd:
    k = 0
    for chunk in r.iter_content(chunk_size=chunk_size):
        # Чтобы быть в курсе процесса, печатаем прогресс, но не каждый раз, а при загрузке каждой 20000 пачки
        if k % 20000 == 0:
            print ('Megabytes downloaded: %s\r' % str(current_size / 1024 / 1024.0))
        k += 1
        
        fd.write(chunk)
        current_size += chunk_size

Megabytes downloaded: 0.0
Megabytes downloaded: 78.125
Megabytes downloaded: 156.25
Megabytes downloaded: 234.375
Megabytes downloaded: 312.5
Megabytes downloaded: 390.625
Megabytes downloaded: 468.75
Megabytes downloaded: 546.875
Megabytes downloaded: 625.0
Megabytes downloaded: 703.125
Megabytes downloaded: 781.25
Megabytes downloaded: 859.375
Megabytes downloaded: 937.5


Кажется, мы только что научились скачивать данные Sentinel-2! Эта процедура включает два основных этапа - получение списка uuid сцен, которые удовлетворяют нашим условиям, а затем загрузка этих uuid.

Разберёмся ещё с важнейшим фильтром - по охвату территории. Мы хотим запрашивать только те сцены, которые покрывают нашу местность. Для этого используется параметр footprint:"Intersects(...)" где вместо ... должна быть геометрия нашей территория в WGS84 и в формате <a href="https://ru.wikipedia.org/wiki/WKT">WKT</a>. Вместо Intersects можно также использовать Contains и IsWithin.

Чтобы добавить такую опцию к запросу, сначала придётся прочитать векторный слой с районом и извлечь его геометрию в формате WKT.

In [79]:
# Это просто сделать с помощью OGR - это мощная библитека для чтения, записи и базовой обработки векторных данных. Есть у неё и интерфейс для Python

# Путь до нашего регона
region_path = 'data/region.geojson'

# открываем его
region_ds = ogr.Open(region_path)

# Достаём из него единственный слой
region_layer = region_ds.GetLayerByIndex(0)

In [80]:
region_layer

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x000002D82AE9C750> >

In [81]:
# Из слоя достаём первый объект
region_feature = region_layer.GetFeature(0)

In [82]:
region_feature

<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x000002D82B20FD80> >

In [83]:
# Из объекта достаём геометрию и представляем её в формате WKT
region_wkt = region_feature.geometry().ExportToWkt()
region_wkt

'POLYGON ((40.2437928352969 47.6688419967706,40.2486554577922 47.6687328446184,40.2521133226778 47.6694241377269,40.2557873041187 47.6685873080605,40.2591371107266 47.6673866159723,40.2644859954715 47.668296233727,40.2648101703045 47.6597815913122,40.2437928352969 47.6546139089538,40.2437928352969 47.6688419967706))'

Такую запись уже можно подставлять в запрос - и нам будут предлагаться только те сцены, которые покрывают или пересекаются с этим полигоном!
Соберём пример полного запроса, ещё с датами и фильтром по облачности

In [84]:
start_date = datetime(2020,5,1).strftime("%Y-%m-%dT%H:%M:%SZ")
end_date = datetime(2020,6,1).strftime("%Y-%m-%dT%H:%M:%SZ")


request_example = 'https://scihub.copernicus.eu/dhus/search?q=(platformname:Sentinel-2) AND (footprint:"Contains(%s)") AND (beginposition:[%s TO %s]) AND (endposition:[%s TO %s])' % (region_wkt, start_date, end_date, start_date, end_date)

In [85]:
request_example

'https://scihub.copernicus.eu/dhus/search?q=(platformname:Sentinel-2) AND (footprint:"Contains(POLYGON ((40.2437928352969 47.6688419967706,40.2486554577922 47.6687328446184,40.2521133226778 47.6694241377269,40.2557873041187 47.6685873080605,40.2591371107266 47.6673866159723,40.2644859954715 47.668296233727,40.2648101703045 47.6597815913122,40.2437928352969 47.6546139089538,40.2437928352969 47.6688419967706)))") AND (beginposition:[2020-05-01T00:00:00Z TO 2020-06-01T00:00:00Z]) AND (endposition:[2020-05-01T00:00:00Z TO 2020-06-01T00:00:00Z])'

In [ ]:
https://scihub.copernicus.eu/dhus/search?q=(platformname:Sentinel-2) AND (footprint:"Intersects(POLYGON ((40.2259091903418 47.6600727131276,40.242442106826 47.6602910534239,40.242442106826 47.6548322720784,40.2249366658428 47.6491545340656,40.2259091903418 47.66)))) AND (beginposition:[2020-05-01T00:00:00Z TO 2020-06-01T00:00:00Z]) AND (endposition:[2020-05-01T00:00:00Z TO 2020-06-01T00:00:00Z])

In [86]:
# Отправляем
r = requests.get(request_example, auth=('spbgeotex','SpbGeoTex0'))

In [87]:
r.status_code

200

In [89]:
r.content[0:1000]

b'<?xml version="1.0" encoding="utf-8"?><feed xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/" xmlns="http://www.w3.org/2005/Atom">\n<title>Sentinels Scientific Data Hub search results for: (platformname:Sentinel-2) AND (footprint:"Contains(POLYGON ((40.2437928352969 47.6688419967706,40.2486554577922 47.6687328446184,40.2521133226778 47.6694241377269,40.2557873041187 47.6685873080605,40.2591371107266 47.6673866159723,40.2644859954715 47.668296233727,40.2648101703045 47.6597815913122,40.2437928352969 47.6546139089538,40.2437928352969 47.6688419967706)))") AND (beginposition:[2020-05-01T00:00:00Z TO 2020-06-01T00:00:00Z]) AND (endposition:[2020-05-01T00:00:00Z TO 2020-06-01T00:00:00Z])</title>\n<subtitle>Displaying 0 to 9 of 12 total results. Request done in 2.489 seconds.</subtitle>\n<updated>2020-11-05T17:43:03.115Z</updated>\n<author>\n<name>Sentinels Scientific Data Hub</name>\n</author>\n<id>https://scihub.copernicus.eu/dhus/search?q=(platformname:Sentinel-2) AND (footprint:"

Теперь, разобравшись во всех основных аспектах формирования запросов, сделаем для себя две красивых функции:
* Функция для поиска данных S2 по начальной и конечной датам, входному файлу с вектором района интереса и ещё нескольким полям
* Функция для загрузки данных по uuid

In [116]:
# Функция поиска
def search_s2 (user, password, start_date, end_date, region_file, cloud_limit=30):
    scenes = []
    
    region_ds = ogr.Open(region_file)
    region_layer = region_ds.GetLayerByIndex(0)
    region_feature = region_layer.GetFeature(0)
    region_wkt = region_feature.geometry().ExportToWkt()
    
    region_rule = '"Contains(%s)"' % region_wkt
    start_date = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_date = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    cloud = '[0 TO %s]' % cloud_limit
    platform = 'Sentinel-2'
    product = 'S2MSI2A'
    
    query = 'https://scihub.copernicus.eu/dhus/search?q=(platformname:%s) AND (producttype:%s) AND (footprint:%s) AND (beginposition:[%s TO %s]) AND (endposition:[%s TO %s]) AND (cloudcoverpercentage:%s)&rows=100' % (platform, product, region_rule, start_date, end_date, start_date, end_date, cloud)
    r = requests.get(query, auth=(user,password))
    
    answer_data = etree.ElementTree(etree.fromstring(r.content)).getroot()
    entries = answer_data.findall('{http://www.w3.org/2005/Atom}entry')
    
    for entry in entries:
        uuid = None
        filename = None
        for child in entry:
            if (child.get('name') == 'uuid'):
                uuid = child.text

            if (child.get('name') == 'filename'):
                filename = child.text
                
        if (not uuid) or (not filename):
            continue
        
        scenes.append({'filename': filename,
                       'uuid': uuid})
    
    return scenes

In [114]:
# Тестируем её
scenes = search_s2(user = 'spbgeotex',
                   password = 'SpbGeoTex0',
                   start_date = datetime(2020,3,1),
                   end_date = datetime(2020,10,1),
                   region_file = 'data/region.geojson')

print (scenes)

[{'filename': 'S2B_MSIL2A_20200927T081719_N0214_R121_T37TEN_20200927T105824.SAFE', 'uuid': '00e58dc5-af52-44a2-b9b3-2407c2378df4'}, {'filename': 'S2A_MSIL2A_20200922T081641_N0214_R121_T37TEN_20200922T101111.SAFE', 'uuid': 'aa259395-af81-48ff-b4f7-e249e39fcdec'}, {'filename': 'S2B_MSIL2A_20200917T081609_N0214_R121_T37TEN_20200917T103537.SAFE', 'uuid': '05d36ab0-4cab-4266-b57a-3ca5ca8206d0'}, {'filename': 'S2A_MSIL2A_20200912T081611_N0214_R121_T37TEN_20200912T104913.SAFE', 'uuid': '095fc8aa-ca9f-43e1-baca-1c9e55763e29'}, {'filename': 'S2B_MSIL2A_20200907T081609_N0214_R121_T37TEN_20200907T110224.SAFE', 'uuid': 'c0fa3f8c-cd43-4191-a14b-d0e8c32d6ad2'}, {'filename': 'S2A_MSIL2A_20200902T081611_N0214_R121_T37TEN_20200902T113611.SAFE', 'uuid': '871c43f6-f5a1-42c0-a5bb-69d10e81b9c3'}, {'filename': 'S2A_MSIL2A_20200823T081611_N0214_R121_T37TEN_20200823T100712.SAFE', 'uuid': '59f402d7-89e1-4c2f-aa65-1ffce153e187'}, {'filename': 'S2B_MSIL2A_20200818T081609_N0214_R121_T37TEN_20200818T110631.SAFE', 

In [115]:
len(scenes)

25

In [117]:
# Функция загрузки
def download_s2_by_uuid (user, password, uuid, destination):
    if os.path.exists(destination):
        print ('Файл уже существует')
        return
    
    current_size = 0
    chunk_size = 4096
    print ('Начинаем загрузку %s' % destination)
    with open(destination, 'wb') as fd:
        k = 0
        for chunk in r.iter_content(chunk_size=chunk_size):
            # Чтобы быть в курсе процесса, печатаем прогресс, но не каждый раз, а при загрузке каждой 20000 пачки
            if k % 20000 == 0:
                print ('Megabytes downloaded: %s\r' % str(current_size / 1024 / 1024.0))
            k += 1

            fd.write(chunk)
            current_size += chunk_size

In [123]:
# И начинаем загрузку всего по списку полученных сцен
for scene in scenes[0:1]:
    download_s2_by_uuid(user = 'spbgeotex',
                        password = 'SpbGeoTex0',
                        uuid = scene['uuid'],
                        destination = os.path.join('data','raw_scenes',scene['filename']))

Начинаем загрузку data\raw_scenes\S2B_MSIL2A_20200927T081719_N0214_R121_T37TEN_20200927T105824.SAFE
Megabytes downloaded: 0.0
